### Raw data retrieval for angelist.co/companies 
Scraped using bs4 and Selenium

In [1]:
import re
import bs4 as bs
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

### Part 1.1: Getting list of urls from "all companies" page

In [2]:
def get_results(tech=None, market=None, stage=None):
    
    url = 'https://angel.co/companies?company_types[]=Startup&company_types[]=Private+Company&company_types[]=SaaS&company_types[]=Mobile+App'
    if tech:
        url = url + '&teches[]=' + tech
    if market:
        market = market.replace(' ', '+')
        url = url + '&markets[]=' + market
    if stage:
        url = url + '&stage=' + stage

    url = url + '&raised[min]=1000&raised[max]=100000000000'
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    
    driver = webdriver.Chrome()

    driver.set_window_size(3000, 1000)
    driver.get(url)
    time.sleep(5)
    
    sortby_raised = driver.find_element_by_xpath("//div[@class='column raised sortable']")

    sortby_raised.click()
    time.sleep(5)
    
    source = driver.page_source
    
    while True:
        try:
            source = driver.page_source
            loadMoreButton = driver.find_element_by_class_name('more').click()
            time.sleep(3)
        except: 
            break
            
    driver.close()

    try:
        soup = bs.BeautifulSoup(source, 'lxml')
        result_list =  soup.find_all('div', {'class': 'results'})[0]
        results = result_list.find_all('div', {'data-_tn': 'companies/row'})
    except:
        print('Could not get results')
        return

    
    return results



In [3]:
def parse_results(results):
    df = pd.DataFrame(columns=['kind', 'link', 'name', 'pitch', 'joined', 
                               'location', 'market', 'size', 'stage', 'raised'],
                  index=[0])
    
    for result in results[1:]:
        try:
            dic = {}
            try:
                dic['kind'] = result.a['data-type']
            except: dic['kind'] = None

            try:
                dic['link'] = result.a['href']
            except: dic['link'] = None

            try:
                dic['name'] = result.a['title']
            except: dic['name'] = None
                
            try:
                dic['website'] = result.find('div', {'data-column': 'website'}).text.split()[1]
            except: dic['website'] = None

            try:
                dic['pitch'] = result.find('div', {'class': 'pitch'}).text
            except: dic['pitch'] = None

            try:
                dic['joined'] = result.find('div', {'data-column': 'joined'}).text.split('Joined')[1].strip()
            except: dic['joined'] = None

            try:
                dic['location'] = result.find('div', {'data-column': 'location'}).text.split('Location')[1].strip()
            except: dic['location'] = None

            try:
                dic['market'] = result.find('div', {'data-column': 'market'}).text.split('Market')[1].strip()
            except: dic['market'] = None

            try:
                dic['size'] = result.find('div', {'data-column': 'company_size'}).text.split()[1]
            except: dic['size'] = None

            try:
                dic['stage'] = result.find('div', {'data-column': 'stage'}).text.split('Stage')[1].strip()
            except: dic['stage'] = None

            try:
                dic['raised'] = result.find('div', {'data-column': 'raised'}).text.split('Raised')[1].strip()
            except: dic['raised'] = None
            
            
            df = df.append(pd.DataFrame(dic, index=[0]))

        except: pass

    df = df.reset_index(drop=True)
    return df



def get_companies(tech=None, market=None, stage=None):
    results = get_results(tech, market, stage)
    if results:
        df = parse_results(results)
        return df

In [4]:
get_companies(tech="HTML5", market="E-Commerce", stage=None)

,kind,link,name,pitch,joined,location,market,size,stage,raised,website
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Startup,https://angel.co/company/path,Path,\nPersonal Networking\n,May ’11,San Francisco,Mobile Commerce,51-200,Acquired,"$66,150,000",path.com
2,Startup,https://angel.co/company/swap-com,Swap.com,\nMaking consignment virtual\n,Mar ’11,Illinois,Online Shopping,11-50,,"$22,500,000",swap.com
3,Startup,https://angel.co/company/zeel,Zeel,\nMassage On Demand\n,Sep ’11,New York City,Mobile Health,51-200,Series A,"$13,200,000",zeel.com
4,Startup,https://angel.co/company/gratafy,Gratafy,"\nExperiential Marketing Platform for Brands, ...",Jul ’11,Seattle,Mobile Commerce,11-50,Closed,"$5,450,000",gratafy.com
...,...,...,...,...,...,...,...,...,...,...,...
131,Startup,https://angel.co/company/r2v,R2V,\nAny screen is your screen\n,Nov ’13,Menlo Park,Mobile Commerce,1-10,Seed,"$5,000",r2v.me
132,Startup,https://angel.co/company/skigit,Skigit,\nTurning Entertaining Consumer Video into Bra...,Feb ’14,Tampa,Social Media,1-10,,"$30,000",mavensoft.org
133,Startup,https://angel.co/company/fountain-greetings,Fountain,\nHandwritten greeting cards. Curated gifts\n,Apr ’14,New York City,Mobile Commerce,1-10,Seed,"$25,000",fountaingreetings.com
134,Startup,https://angel.co/company/marketcloud,Marketcloud,\nMobile-first eCommerce backend as a service\n,Sep ’14,Bologna,Developer APIs,1-10,Seed,"$25,000",marketcloud.it


In [ ]:
import datetime
datetime.datetime.now()

In [ ]:
filename="angelist_"+str(datetime.datetime.now())[:-7].replace(":","_").replace(" ","_")+".csv"
filename

In [ ]:
filename="angelist_"+str(datetime.datetime.now())[:-7].replace(":","_").replace(" ","_")+".csv"

techs = ['Python', 'Javascript', 'HTML5', 'Java', 'CSS', None]

markets = ["E-Commerce", "Education", "Enterprise Software", "Games", "Healthcare", "Mobile", "Big Data", "AI", "ML", None]


df = pd.DataFrame()

for tech in techs:
    for market in markets:
        print(tech, market)

        try:
            companies = get_companies(tech, market, None)
#                 companies['query'] = query
#                 companies['type_'] = type_
#                 companies['stage'] = stage
#                 companies['tech'] = tech
            print('- Got companies')
        except Exception as e: 
            companies = pd.DataFrame()
            print('!! Could not get companies')
            print(e)
            pass

        try:
            if df.empty:
                df = companies
            else:
                df = df.append(companies)
            print("- Added {} rows".format(companies.shape[0]))
            print("- Total rows: {}".format(df.shape[0]))
        except:
            print('!! Unable to append new data.')
            pass

    df = df.reset_index(drop=True)
    df.to_csv(filename)
    print("-"*50)

            
df = df.reset_index(drop=True)
df.to_csv(filename)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df_unique = df.drop_duplicates(subset ="link" ) 

In [ ]:
df_unique.shape

In [ ]:
df_unique.sort_values("name")

In [ ]:
df_unique.to_csv('angelist_unique_4261.csv')